In [3]:
#instalo/actualizo
#! pip3 install --upgrade google-cloud-aiplatform


In [4]:
import re
import pandas as pd
from google.cloud import bigquery
#import vertexai 
#from vertexai.generative_models import GenerationConfig, GenerativeModel, Image, Part, Content
#import vertexai.preview.generative_models as generative_models 
import sys
import os
from fuzzywuzzy import process, fuzz

In [5]:
import os 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\\Users\\flori\\OneDrive\\Desktop\\LDP\\ldp-data-genai-producto-b871ebffc681.json"

In [6]:
# Crear un cliente de BigQuery con el proyecto especificado
client = bigquery.Client(project='ldp-data-wyscout')


PARA EMPEZAR ME TRAIGO LAS TABLAS DE TRANSFERENCIAS DE TFMKT Y WYSCOUT

In [7]:


# Consulta SQL
query2 = """SELECT * FROM ldp-data-wyscout.transfer_market.players_transfers_curated"""

try:
    # Ejecutar la consulta y obtener los resultados en un DataFrame
    df1 = client.query(query2).to_dataframe()
    print(df1.head())
except Exception as e:
    print(f"An error occurred: {e}")


  row_ind       player_name season transfer_date      team_from  \
0      19      Noah Sadaoui  04/05    2005-01-01   Wydad AC U19   
1      12       Atınç Nukan  05/06    2006-02-13  K.Cekmece Yth   
2      15  Heung-Fai Cheung  99/00    1999-07-01   Instant-Dict   
3       9    Hassan Maatouk  03/04    2003-07-01   RW Essen U17   
4      14       Deniz Kadah  05/06    2005-07-01  Rotenburg U19   

          team_to market_value transfer_value contract_expiry  \
0    NYRB Academy           NA              0              NA   
1    Besiktas Yth           NA              0              NA   
2         Kitchee           NA              0              NA   
3  Al-Ahed FC U20           NA             NA              NA   
4  Rotenburger SV           NA             NA              NA   

                                          player_url  query_date  \
0  https://www.transfermarkt.com/noah-sadaoui/pro...  2024-09-08   
1  https://www.transfermarkt.com/atinc-nukan/prof...  2024-09-11   
2 

In [8]:
#df1=pd.read_csv('tfmkt.csv')

In [9]:
df1.head(20)

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id
0,19,Noah Sadaoui,04/05,2005-01-01,Wydad AC U19,NYRB Academy,NA,0,NA,https://www.transfermarkt.com/noah-sadaoui/pro...,2024-09-08,2024-09-07 14:32:10+00:00,transfermarkt/player_transfers/ players_transf...,331478,https://www.transfermarkt.com/fc-goa/startseit...,45321,2005010145321331478
1,12,Atınç Nukan,05/06,2006-02-13,K.Cekmece Yth,Besiktas Yth,NA,0,NA,https://www.transfermarkt.com/atinc-nukan/prof...,2024-09-11,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,149516,https://www.transfermarkt.com/goztepe/startsei...,1467,200602131467149516
2,15,Heung-Fai Cheung,99/00,1999-07-01,Instant-Dict,Kitchee,NA,0,NA,https://www.transfermarkt.com/heung-fai-cheung...,2024-09-08,2024-09-07 14:31:16+00:00,transfermarkt/player_transfers/ players_transf...,835001,https://www.transfermarkt.com/shatin/startseit...,34344,1999070134344835001
3,9,Hassan Maatouk,03/04,2003-07-01,RW Essen U17,Al-Ahed FC U20,NA,NA,NA,https://www.transfermarkt.com/hassan-maatouk/p...,2024-09-07,2024-09-07 14:31:16+00:00,transfermarkt/player_transfers/ players_transf...,98579,https://www.transfermarkt.com/al-ansar/startse...,2468,20030701246898579
4,14,Deniz Kadah,05/06,2005-07-01,Rotenburg U19,Rotenburger SV,NA,NA,NA,https://www.transfermarkt.com/deniz-kadah/prof...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,50413,https://www.transfermarkt.com/altay-sk/startse...,2375,20050701237550413
5,9,Yong-rae Lee,02/03,2002-10-07,Yuseong BST HS,Metz Youth,NA,NA,NA,https://www.transfermarkt.com/yong-rae-lee/pro...,2024-09-08,2024-09-07 14:32:10+00:00,transfermarkt/player_transfers/ players_transf...,172007,https://www.transfermarkt.com/daegu-fc/startse...,6504,200210076504172007
6,23,Andy Lonergan,02/03,2002-07-01,Preston U18,Preston,NA,NA,NA,https://www.transfermarkt.com/andy-lonergan/pr...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,14044,https://www.transfermarkt.com/fc-everton/start...,29,200207012914044
7,15,Yuto Nakamura,04/05,2005-04-01,Urawa Reds U18,Aoyama Gakuin U,NA,0,NA,https://www.transfermarkt.com/yuto-nakamura/pr...,2024-09-08,2024-09-07 14:32:10+00:00,transfermarkt/player_transfers/ players_transf...,129173,https://www.transfermarkt.com/lee-man/startsei...,61336,2005040161336129173
8,16,Alfred Finnbogason,05/06,2006-03-15,Breidablik U19,Breidablik,NA,NA,NA,https://www.transfermarkt.com/alfred-finnbogas...,2024-09-07,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,90024,https://www.transfermarkt.com/lyngby-bk/starts...,369,2006031536990024
9,20,Ruben Ligeon,00/01,2000-07-01,OSV Youth,Ajax Youth,NA,0,NA,https://www.transfermarkt.com/ruben-ligeon/pro...,2024-09-11,2024-09-07 14:32:10+00:00,transfermarkt/player_transfers/ players_transf...,111185,https://www.transfermarkt.com/nac-breda/starts...,132,20000701132111185


In [10]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1101741 entries, 0 to 1101740
Data columns (total 17 columns):
 #   Column              Non-Null Count    Dtype              
---  ------              --------------    -----              
 0   row_ind             1101741 non-null  object             
 1   player_name         1101741 non-null  object             
 2   season              1101741 non-null  object             
 3   transfer_date       1101741 non-null  object             
 4   team_from           1101741 non-null  object             
 5   team_to             1101741 non-null  object             
 6   market_value        1101741 non-null  object             
 7   transfer_value      1101741 non-null  object             
 8   contract_expiry     1101741 non-null  object             
 9   player_url          1101741 non-null  object             
 10  query_date          1101741 non-null  dbdate             
 11  timestamp           1101741 non-null  datetime64[us, UTC]
 12  

In [11]:
#dftext = df1.to_csv('tfmkt.csv', index=False)

In [12]:
# Contar valores nulos en df1
null_counts_df1 = df1.isnull().sum()
print("Valores nulos en df1:")
print(null_counts_df1)



Valores nulos en df1:
row_ind                   0
player_name               0
season                    0
transfer_date             0
team_from                 0
team_to                   0
market_value              0
transfer_value            0
contract_expiry           0
player_url                0
query_date                0
timestamp                 0
blob                  96968
player_id                 1
teams_url                 1
team_id                   1
player_transfer_id        1
dtype: int64


In [13]:
###FILTRO LA DE DICT TEAMS POR team_type="parent" and category="default" and gender = 'male' Y ADEMAS POR LOS PAISES QUE ESTAN EN LA DE CAPOLOGY PARA ACHICAR REGISTROS

query = """select * from ldp-data-wyscout.wyscout_curated.transfer_con_nombre;
 """

try:
    # Ejecutar la consulta y obtener los resultados en un DataFrame                     ###domestic_competition_name
    df2 = client.query(query).to_dataframe()
    print(df2.head())
except Exception as e:
    print(f"An error occurred: {e}")

                nombre  transferId  fromTeamId                fromTeamName  \
0  Herman Oen Paulsrud     2252070       24654        Grorud Idrettslag II   
1        Lucas Poulsen     2352906        7463                BK Frem 1886   
2         Osama Housni     2252131        7292                  Holmlia SK   
3         Ahmad Shaban     2376923       13294              Hapoel Acre FC   
4   Ryan Pereira Costa     2352994       23709  Os Belenenses SAD Under 19   

   toTeamId       toTeamName active   startDate     endDate        type value  \
0      7345        Grorud IL      0  2024-03-22        None    Transfer     0   
1     34555        Sundby BK      0  2024-08-01        None    Transfer     0   
2      7354    Nordstrand IF      0  2023-07-28  2024-03-05    Transfer     0   
3     13364  Hapoel Afula FC      0  2024-07-01        None    Transfer     0   
4      <NA>             None      0  2024-07-30        None  Free Agent     0   

  currency announceDate  playerId  
0      E

In [14]:
# Contar valores nulos en df2
null_counts_df2 = df2.isnull().sum()
print("Valores nulos en df2:")
print(null_counts_df2)

Valores nulos en df2:
nombre              77
transferId           0
fromTeamId      593112
fromTeamName    198586
toTeamId        293553
toTeamName      229523
active               0
startDate         1026
endDate         696105
type                55
value                0
currency          1395
announceDate         0
playerId             0
dtype: int64


In [15]:
# Filtrar las filas donde fromTeamId es nulo
null_fromTeamId_rows = df2[df2['fromTeamId'].isnull()]

# Mostrar las filas con valores nulos en fromTeamId
null_fromTeamId_rows.head(30)

,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
6,Henrik Bertelsen,2251733,<NA>,,7348,Jerv,0,2024-03-06,None,Free Transfer,0,EUR,,1061870
7,Niklas Schmidt-Løvlund,2252025,<NA>,,7143,Follo,0,2024-04-01,None,Free Transfer,0,EUR,,1061910
8,Linus Kahisalo,2251609,<NA>,,4812,Gnistan,0,2023-07-01,None,Free Transfer,0,EUR,,1061850
9,Michael Famuyegun,2252258,<NA>,,74803,Broad City,0,2023-10-01,None,Free Transfer,0,EUR,,1061977
10,Elias Leonel Galvez,2251613,<NA>,,12162,CSD Liniers,0,2024-03-01,None,Free Transfer,0,EUR,,1061852
11,Sanel Ibisevic,2251973,<NA>,,7352,Follo FK II,0,2015-05-05,2017-08-24,Free Transfer,0,EUR,,1061902
12,Pierre Bastan Gomes,2252421,<NA>,,75651,Ogre United,0,2024-01-01,None,Free Transfer,0,EUR,,1062007
13,Einars Sidrevics,2252414,<NA>,,75651,Ogre United,0,2024-01-01,None,Free Transfer,0,EUR,,1062003
14,Anrijs Nelsons,2252429,<NA>,,75651,Ogre United,0,2024-01-01,None,Free Transfer,0,EUR,,1062012
15,Emils Masulis,2252430,<NA>,,75651,Ogre United,0,2024-04-03,None,Free Transfer,0,EUR,,1062013


In [16]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1933826 entries, 0 to 1933825
Data columns (total 14 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   nombre        object
 1   transferId    Int64 
 2   fromTeamId    Int64 
 3   fromTeamName  object
 4   toTeamId      Int64 
 5   toTeamName    object
 6   active        object
 7   startDate     object
 8   endDate       object
 9   type          object
 10  value         object
 11  currency      object
 12  announceDate  object
 13  playerId      Int64 
dtypes: Int64(4), object(10)
memory usage: 213.9+ MB


In [17]:

# Eliminar filas con valores nulos en df2
df2_clean = df2.dropna()

print("\nDataFrame df2_clean:")
print(df2_clean.head())


DataFrame df2_clean:
                              nombre  transferId  fromTeamId  \
2                       Osama Housni     2252131        7292   
34                  Danila Strelchuk     1599722       30833   
37                      Edonis Mulaj      993058        7196   
38                      Nikita Gubin     1542161       13951   
39  Daniel Shaquille Jabari Phillips     1732160       22981   

            fromTeamName  toTeamId               toTeamName active  \
2             Holmlia SK      7354            Nordstrand IF      0   
34    FK Arsenal Tula II     13905          FK Arsenal Tula      0   
37  Vålerenga Fotball II     33362  Idrettsforeningen Ready      0   
38      FK Lada Tolyatti     11127           FC Dinamo-Auto      0   
39   Watford FC Under 21      1836  Hemel Hempstead Town FC      0   

     startDate     endDate      type value currency announceDate  playerId  
2   2023-07-28  2024-03-05  Transfer     0      EUR                1061932  
34  2021-07-01  20

In [18]:
df2_clean.head(20)

,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
2,Osama Housni,2252131,7292,Holmlia SK,7354,Nordstrand IF,0,2023-07-28,2024-03-05,Transfer,0,EUR,,1061932
34,Danila Strelchuk,1599722,30833,FK Arsenal Tula II,13905,FK Arsenal Tula,0,2021-07-01,2022-02-17,Transfer,0,EUR,,685462
37,Edonis Mulaj,993058,7196,Vålerenga Fotball II,33362,Idrettsforeningen Ready,0,2022-04-06,2023-02-01,Unknown,0,EUR,,685593
38,Nikita Gubin,1542161,13951,FK Lada Tolyatti,11127,FC Dinamo-Auto,0,2022-04-08,2022-07-19,Transfer,0,EUR,,685604
39,Daniel Shaquille Jabari Phillips,1732160,22981,Watford FC Under 21,1836,Hemel Hempstead Town FC,0,2019-08-22,2019-12-31,Loan,0,EUR,,685477
40,Matías Jesús Araya Gaytan,1870423,12308,CD Copiapó,58687,Club Deportivo Provincial Oval,0,2023-03-10,2023-09-09,Transfer,0,EUR,,685547
42,Raúl da Silva,992963,28013,Grebbestads IF,30574,Holbæk Bold- & Idrætsforening,0,2020-08-01,2021-04-30,Transfer,0,EUR,,685533
43,Marco Mannhardt,1821409,2461,TSV 1860 München,2592,TSV 1860 München II,0,2022-07-01,2023-01-01,Transfer,0,EUR,,685571
44,Gábor Vágó,994089,10207,Kecskeméti TE,24939,MOL Fehérvár FC Under 19,0,2019-08-15,2021-02-01,Transfer,0,EUR,,685597
45,Léider Andrés Riascos Suárez,2310597,12396,Club Deportes Tolima S.A.,12411,Club Deportivo La Equidad Segu,0,2024-07-16,2025-07-30,Loan,0,EUR,,685540


In [19]:
df2_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 854637 entries, 2 to 1933825
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   nombre        854637 non-null  object
 1   transferId    854637 non-null  Int64 
 2   fromTeamId    854637 non-null  Int64 
 3   fromTeamName  854637 non-null  object
 4   toTeamId      854637 non-null  Int64 
 5   toTeamName    854637 non-null  object
 6   active        854637 non-null  object
 7   startDate     854637 non-null  object
 8   endDate       854637 non-null  object
 9   type          854637 non-null  object
 10  value         854637 non-null  object
 11  currency      854637 non-null  object
 12  announceDate  854637 non-null  object
 13  playerId      854637 non-null  Int64 
dtypes: Int64(4), object(10)
memory usage: 101.1+ MB


In [20]:
# Ordenar df1 alfabéticamente por la columna 'player_name'
df1_sorted = df1.sort_values(by='player_name')
print("df1 ordenado alfabéticamente por 'player_name':")




df1 ordenado alfabéticamente por 'player_name':


In [21]:
df1_sorted.head(50)

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id
928313,1,- Stewart,23/24,2023-07-01,Gala Fairydean,Without Club,NA,NA,NA,https://www.transfermarkt.com/-stewart/profil/...,2024-09-11,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,1092926,https://www.transfermarkt.com/gala-fairydean-r...,11379,20230701113791092926
557601,1,. Bana,20/21,2020-07-01,Own Youth,Desp. Huíla,NA,NA,NA,https://www.transfermarkt.com/-bana/profil/spi...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,887419,https://www.transfermarkt.com/desportivo-da-hu...,39419,2020070139419887419
890458,1,A Mohamed Liyaakath,23/24,2024-02-13,Without Club,G. Kerala B,25000,NA,NA,https://www.transfermarkt.com/a-mohamed-liyaak...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,842536,https://www.transfermarkt.com/chennaiyin-fc-ii...,67620,2024021367620842536
1019155,2,A Mohamed Liyaakath,23/24,2023-06-01,Chennaiyin FC,Without Club,25000,NA,NA,https://www.transfermarkt.com/a-mohamed-liyaak...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,842536,https://www.transfermarkt.com/chennaiyin-fc-ii...,67620,2023060167620842536
819361,3,A Mohamed Liyaakath,21/22,2021-11-01,Indian Arrows,Chennaiyin FC,25000,0,NA,https://www.transfermarkt.com/a-mohamed-liyaak...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,842536,https://www.transfermarkt.com/chennaiyin-fc-ii...,67620,2021110167620842536
717300,4,A Mohamed Liyaakath,20/21,2020-11-01,Chennaiyin U16,Indian Arrows,NA,0,NA,https://www.transfermarkt.com/a-mohamed-liyaak...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,842536,https://www.transfermarkt.com/chennaiyin-fc-ii...,67620,2020110167620842536
954733,1,A Ranjethkumar,23/24,2023-06-01,Chennaiyin II,Without Club,NA,NA,NA,https://www.transfermarkt.com/a-ranjethkumar/p...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,1118356,https://www.transfermarkt.com/chennaiyin-fc-ii...,67620,20230601676201118356
607066,5,A San,19/20,2019-12-31,HCMC FC,HAGL FC,NA,NA,NA,https://www.transfermarkt.com/a-san/profil/spi...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,560526,https://www.transfermarkt.com/becamex-binh-duo...,10756,2019123110756560526
972726,3,A San,22/23,2022-11-30,CAHN FC,HAGL FC,NA,NA,2022-11-30,https://www.transfermarkt.com/a-san/profil/spi...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,560526,https://www.transfermarkt.com/becamex-binh-duo...,10756,2022113010756560526
533988,4,A San,19/20,2020-02-11,HAGL FC,CAND FC,NA,NA,NA,https://www.transfermarkt.com/a-san/profil/spi...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,560526,https://www.transfermarkt.com/becamex-binh-duo...,10756,2020021110756560526


In [22]:
# Ordenar df2 alfabéticamente por la columna 'nombre'
df2_sorted = df2_clean.sort_values(by='nombre')
print("\ndf2 ordenado alfabéticamente por 'nombre':")



df2 ordenado alfabéticamente por 'nombre':


In [23]:
df2_sorted.head(50)

,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
547185,Adrian Chelinski Chelinski,1348913,13843,KS SKRA Częstochowa,13522,Radomsko,0,2022-07-01,2023-06-30,Loan,0,EUR,,756213
874621,Ahmad Reza Jalali,809754,58935,Foolad FC U19,10769,Foolad Khuzestan FC,0,2018-07-01,2019-07-01,Transfer,0,EUR,,541746
836140,Ahmad Reza Jalali,1663697,10769,Foolad Khuzestan FC,26651,Shahr Khodrou FC,0,2019-07-01,2019-10-23,Transfer,0,EUR,,541746
836207,Ahmad Reza Jalali,1663699,26651,Shahr Khodrou FC,10769,Foolad Khuzestan FC,0,2019-10-23,2022-07-01,Transfer,0,EUR,,541746
819715,Ahmad Reza Jalali,1663700,10769,Foolad Khuzestan FC,10833,Aluminium Arak FC,0,2022-07-01,2023-01-14,Transfer,0,EUR,,541746
93261,Aidan Jun Valcarcel Daniels,737744,14995,Toronto,27049,Ottawa Fury,0,2019-03-06,2019-11-30,Loan,0,EUR,,425780
8698,Aidan Jun Valcarcel Daniels,737745,27049,Ottawa Fury,14995,Toronto,0,2019-11-30,2020-01-01,Back from Loan,0,EUR,,425780
93299,Aidan Jun Valcarcel Daniels,1146531,33171,Colorado Springs Switchbacks F,28728,Oklahoma City Energy FC,0,2021-01-26,2022-01-19,Transfer,0,EUR,,425780
93197,Aidan Jun Valcarcel Daniels,863944,14995,Toronto,33171,Colorado Springs,0,2020-01-01,2021-01-26,Transfer,0,EUR,,425780
8646,Aidan Jun Valcarcel Daniels,598022,14995,Toronto,33175,Toronto II,0,2018-11-30,2019-03-06,Back from Loan,0,EUR,,425780


In [24]:
#df2=pd.read_csv('wyscout.csv')

In [25]:
# Reemplazar NaN con un valor por defecto, por ejemplo, -1 o 0
#df2['fromTeamId'] = df2['fromTeamId'].fillna(0).astype(int)
#df2['toTeamId'] = df2['toTeamId'].fillna(0).astype(int)


In [26]:
df2.head()

,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
0,Herman Oen Paulsrud,2252070,24654,Grorud Idrettslag II,7345,Grorud IL,0,2024-03-22,None,Transfer,0,EUR,,1061921
1,Lucas Poulsen,2352906,7463,BK Frem 1886,34555,Sundby BK,0,2024-08-01,None,Transfer,0,EUR,,1061847
2,Osama Housni,2252131,7292,Holmlia SK,7354,Nordstrand IF,0,2023-07-28,2024-03-05,Transfer,0,EUR,,1061932
3,Ahmad Shaban,2376923,13294,Hapoel Acre FC,13364,Hapoel Afula FC,0,2024-07-01,None,Transfer,0,EUR,,1061976
4,Ryan Pereira Costa,2352994,23709,Os Belenenses SAD Under 19,<NA>,None,0,2024-07-30,None,Free Agent,0,EUR,,1061882


In [27]:
#dftext2 = df2.to_csv('wyscout.csv',index=False)

In [28]:
# Tomar las primeras 1000 filas de df1_sorted y df2_sorted
df1_sorted_top1000 = df1_sorted.head(1000)
df2_sorted_top1000 = df2_sorted.head(1000)


In [29]:
# Tomar las primeras 5000 filas de df1_sorted y df2_sorted
df1_sorted_top5000 = df1_sorted.head(5000)
df2_sorted_top5000 = df2_sorted.head(5000)

# Mostrar las primeras filas para verificar
print(df1_sorted_top5000.head())
print(df2_sorted_top5000.head())

        row_ind          player_name season transfer_date       team_from  \
928313        1            - Stewart  23/24    2023-07-01  Gala Fairydean   
557601        1               . Bana  20/21    2020-07-01       Own Youth   
890458        1  A Mohamed Liyaakath  23/24    2024-02-13    Without Club   
1019155       2  A Mohamed Liyaakath  23/24    2023-06-01   Chennaiyin FC   
819361        3  A Mohamed Liyaakath  21/22    2021-11-01   Indian Arrows   

               team_to market_value transfer_value contract_expiry  \
928313    Without Club           NA             NA              NA   
557601     Desp. Huíla           NA             NA              NA   
890458     G. Kerala B        25000             NA              NA   
1019155   Without Club        25000             NA              NA   
819361   Chennaiyin FC        25000              0              NA   

                                                player_url  query_date  \
928313   https://www.transfermarkt.com/-st

In [30]:
import pandas as pd
import unicodedata
from fuzzywuzzy import fuzz

In [31]:

def normalize_text(text):
    if isinstance(text, str):
        # Convertir a minúsculas, normalizar acentos y eliminar exceso de espacios
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        text = ' '.join(text.lower().strip().split())
        return text
    return text

def simplify_name(name):
    parts = name.split()
    # Retorna sólo el primer y último nombre
    return ' '.join([parts[0], parts[-1]]) if len(parts) > 1 else name

def simplify_club_name(club_name):
    # Convierte a minúsculas y elimina palabras comunes (ajusta según tus necesidades)
    ignore_words = {'fc', 'club', 'de', 'the'}
    parts = club_name.lower().split()
    return ' '.join([word for word in parts if word not in ignore_words])

# Cargar los archivos CSV en dataframes
#df1_sorted_top1000 = pd.read_csv('archivo1.csv')
#df2_sorted_top5000 = pd.read_csv('archivo2.csv')

# Normalizar y simplificar columnas relevantes
df1_sorted_top1000['player_name'] = df1_sorted_top1000['player_name'].apply(normalize_text).apply(simplify_name)
df1_sorted_top1000['team_from'] = df1_sorted_top1000['team_from'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top1000['team_to'] = df1_sorted_top1000['team_to'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top1000['transfer_date'] = pd.to_datetime(df1_sorted_top1000['transfer_date'], errors='coerce')

df2_sorted_top1000['nombre'] = df2_sorted_top1000['nombre'].apply(normalize_text).apply(simplify_name)
df2_sorted_top1000['fromTeamName'] = df2_sorted_top1000['fromTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top1000['toTeamName'] = df2_sorted_top1000['toTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top1000['startDate'] = pd.to_datetime(df2_sorted_top1000['startDate'], errors='coerce')

# Crear un ID temporal basado en mes y año
df1_sorted_top1000['ID'] = df1_sorted_top1000.apply(lambda row: f"{row['team_from']}_{row['team_to']}_{row['transfer_date'].year}_{row['transfer_date'].month}", axis=1)
df2_sorted_top1000['ID'] = df2_sorted_top1000.apply(lambda row: f"{row['fromTeamName']}_{row['toTeamName']}_{row['startDate'].year}_{row['startDate'].month}", axis=1)

# Unión basada en IDs aproximados y similitud de nombres
merged_rows = []

for idx1, row1 in df1_sorted_top1000.iterrows():
    for idx2, row2 in df2_sorted_top1000.iterrows():
        if row1['ID'] == row2['ID']:
            # Usar similitud fuzzywuzzy para nombre del jugador
            name_similarity = fuzz.partial_ratio(row1['player_name'], row2['nombre'])
            # Verifica que la similitud supere un umbral (ajustable según necesidad)
            if name_similarity > 80:
                merged_rows.append(row1.to_dict() | row2.to_dict())

# Crear un dataframe con los resultados de la unión
merged_df22 = pd.DataFrame(merged_rows)

# Ver los resultados
print(merged_df22.head())

# Guardar a un archivo CSV
merged_df22.to_csv('archivo_unidomil.csv', index=False)


C:\Users\flori\AppData\Local\Temp\ipykernel_18016\4026337262.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sorted_top1000['player_name'] = df1_sorted_top1000['player_name'].apply(normalize_text).apply(simplify_name)
C:\Users\flori\AppData\Local\Temp\ipykernel_18016\4026337262.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sorted_top1000['team_from'] = df1_sorted_top1000['team_from'].apply(normalize_text).apply(simplify_club_name)
C:\Users\flori\AppData\Local\Temp\ipykernel_18016\4026337

  row_ind           player_name season transfer_date       team_from  \
0       3            aj cochran  20/21    2021-02-17  phoenix rising   
1       2         aaren d'silva  20/21    2020-10-28          goa ii   
2      15  aaron amadi-holloway  11/12    2012-01-01       bath city   
3       6  aaron amadi-holloway  20/21    2020-10-17   brisbane roar   
4      16  aaron amadi-holloway  11/12    2011-11-24    bristol city   

          team_to market_value transfer_value contract_expiry  \
0     indy eleven       250000              0              NA   
1             goa        75000             NA              NA   
2    bristol city        50000             NA              NA   
3  sc east bengal        2e+05              0              NA   
4       bath city        50000             NA      2012-06-30   

                                          player_url  ... toTeamId  \
0  https://www.transfermarkt.com/aj-cochran/profi...  ...    27170   
1  https://www.transfermarkt.com/aar

In [32]:
merged_df2.head()

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,...,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
0,3,aj cochran,20/21,2021-02-17,phoenix rising,indy eleven,250000,0,NA,https://www.transfermarkt.com/aj-cochran/profi...,...,27170,indy eleven,0,2021-02-17,2023-02-02,Transfer,0,EUR,,300553
1,2,aaren d'silva,20/21,2020-10-28,goa ii,goa,75000,NA,NA,https://www.transfermarkt.com/aaren-dsilva/pro...,...,31884,goa,0,2020-10-28,2021-06-18,Transfer,0,EUR,,703579
2,15,aaron amadi-holloway,11/12,2012-01-01,bath city,bristol city,50000,NA,NA,https://www.transfermarkt.com/aaron-amadi-holl...,...,1655,bristol city,0,2012-01-01,2014-02-12,Back from Loan,0,EUR,,10612
3,6,aaron amadi-holloway,20/21,2020-10-17,brisbane roar,sc east bengal,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,...,15036,sc east bengal,0,2020-10-17,2021-07-23,Free Transfer,0,EUR,,10612
4,16,aaron amadi-holloway,11/12,2011-11-24,bristol city,bath city,50000,NA,2012-06-30,https://www.transfermarkt.com/aaron-amadi-holl...,...,1728,bath city,0,2011-11-24,2012-01-01,Loan,0,,2011-11-26,10612


: 

In [ ]:


# Función para normalizar y simplificar nombres de jugadores y equipos
def normalize_text(text):
    if isinstance(text, str):
        # Convertir a minúsculas, normalizar acentos y eliminar exceso de espacios
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        text = ' '.join(text.lower().strip().split())
        return text
    return text

def simplify_name(name):
    parts = name.split()
    # Retorna sólo el primer y último nombre
    return ' '.join([parts[0], parts[-1]]) if len(parts) > 1 else name

def simplify_club_name(club_name):
    # Convierte a minúsculas y elimina palabras comunes (ajusta según tus necesidades)
    ignore_words = {'fc', 'club', 'de', 'the'}
    parts = club_name.lower().split()
    return ' '.join([word for word in parts if word not in ignore_words])

# Cargar los archivos CSV en dataframes
#df1_sorted_top5000 = pd.read_csv('archivo1.csv')
#df2_sorted_top5000 = pd.read_csv('archivo2.csv')

# Normalizar y simplificar columnas relevantes
df1_sorted_top5000['player_name'] = df1_sorted_top5000['player_name'].apply(normalize_text).apply(simplify_name)
df1_sorted_top5000['team_from'] = df1_sorted_top5000['team_from'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top5000['team_to'] = df1_sorted_top5000['team_to'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top5000['transfer_date'] = pd.to_datetime(df1_sorted_top5000['transfer_date'], errors='coerce')

df2_sorted_top5000['nombre'] = df2_sorted_top5000['nombre'].apply(normalize_text).apply(simplify_name)
df2_sorted_top5000['fromTeamName'] = df2_sorted_top5000['fromTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top5000['toTeamName'] = df2_sorted_top5000['toTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top5000['startDate'] = pd.to_datetime(df2_sorted_top5000['startDate'], errors='coerce')

# Crear un ID temporal basado en mes y año
df1_sorted_top5000['ID'] = df1_sorted_top5000.apply(lambda row: f"{row['team_from']}_{row['team_to']}_{row['transfer_date'].year}_{row['transfer_date'].month}", axis=1)
df2_sorted_top5000['ID'] = df2_sorted_top5000.apply(lambda row: f"{row['fromTeamName']}_{row['toTeamName']}_{row['startDate'].year}_{row['startDate'].month}", axis=1)

# Unión basada en IDs aproximados y similitud de nombres
merged_rows = []

for idx1, row1 in df1_sorted_top5000.iterrows():
    for idx2, row2 in df2_sorted_top5000.iterrows():
        if row1['ID'] == row2['ID']:
            # Usar similitud fuzzywuzzy para nombre del jugador
            name_similarity = fuzz.partial_ratio(row1['player_name'], row2['nombre'])
            # Verifica que la similitud supere un umbral (ajustable según necesidad)
            if name_similarity > 80:
                merged_rows.append(row1.to_dict() | row2.to_dict())

# Crear un dataframe con los resultados de la unión
merged_df2 = pd.DataFrame(merged_rows)

# Ver los resultados
print(merged_df2.head())

# Guardar a un archivo CSV
merged_df2.to_csv('archivo_unido2.csv', index=False)


C:\Users\flori\AppData\Local\Temp\ipykernel_18016\4030022836.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sorted_top5000['player_name'] = df1_sorted_top5000['player_name'].apply(normalize_text).apply(simplify_name)
C:\Users\flori\AppData\Local\Temp\ipykernel_18016\4030022836.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sorted_top5000['team_from'] = df1_sorted_top5000['team_from'].apply(normalize_text).apply(simplify_club_name)
C:\Users\flori\AppData\Local\Temp\ipykernel_18016\4030022

In [19]:


# Funciones auxiliares para la normalización
def normalize_text(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        return ' '.join(text.lower().strip().split())
    return text

def simplify_name(name):
    parts = name.split()
    return ' '.join([parts[0], parts[-1]]) if len(parts) > 1 else name

def simplify_club_name(club_name):
    ignore_words = {'fc', 'club', 'de', 'the'}
    return ' '.join([word for word in club_name.lower().split() if word not in ignore_words])

# Normalizar y simplificar columnas relevantes
df1_sorted_top5000['player_name'] = df1_sorted_top5000['player_name'].apply(normalize_text).apply(simplify_name)
df1_sorted_top5000['team_from'] = df1_sorted_top5000['team_from'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top5000['team_to'] = df1_sorted_top5000['team_to'].apply(normalize_text).apply(simplify_club_name)
df1_sorted_top5000['transfer_date'] = pd.to_datetime(df1_sorted_top5000['transfer_date'], errors='coerce')

df2_sorted_top5000['nombre'] = df2_sorted_top5000['nombre'].apply(normalize_text).apply(simplify_name)
df2_sorted_top5000['fromTeamName'] = df2_sorted_top5000['fromTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top5000['toTeamName'] = df2_sorted_top5000['toTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted_top5000['startDate'] = pd.to_datetime(df2_sorted_top5000['startDate'], errors='coerce')

# Crear un ID temporal basado en mes y año
df1_sorted_top5000['ID'] = df1_sorted_top5000.apply(lambda row: f"{row['team_from']}_{row['team_to']}_{row['transfer_date'].year}_{row['transfer_date'].month}", axis=1)
df2_sorted_top5000['ID'] = df2_sorted_top5000.apply(lambda row: f"{row['fromTeamName']}_{row['toTeamName']}_{row['startDate'].year}_{row['startDate'].month}", axis=1)

# Primaria unión por ID
initial_merge = pd.merge(df1_sorted_top5000, df2_sorted_top5000, on='ID', suffixes=('_df1', '_df2'))

# Unión final basada en similitud
merged_rows = []

for _, merged_row in initial_merge.iterrows():
    name_similarity = fuzz.partial_ratio(merged_row['player_name'], merged_row['nombre'])
    if name_similarity > 80:
        merged_rows.append(merged_row)

# Crear un dataframe final con los resultados
merged_df = pd.DataFrame(merged_rows)

# Ver los resultados
print(merged_df.head())

# Guardar el resultado en un archivo CSV
merged_df.to_csv('archivo_unido.csv', index=False)


C:\Users\flori\AppData\Local\Temp\ipykernel_19560\1654267820.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sorted_top5000['player_name'] = df1_sorted_top5000['player_name'].apply(normalize_text).apply(simplify_name)
C:\Users\flori\AppData\Local\Temp\ipykernel_19560\1654267820.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sorted_top5000['team_from'] = df1_sorted_top5000['team_from'].apply(normalize_text).apply(simplify_club_name)
C:\Users\flori\AppData\Local\Temp\ipykernel_19560\1654267

AttributeError: 'NoneType' object has no attribute 'lower'

In [31]:

pd.set_option('display.max_columns', None)

In [40]:
merged_df.head(20)

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id,ID,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
0,3,aj cochran,20/21,2021-02-17,phoenix rising,indy eleven,250000,0,NA,https://www.transfermarkt.com/aj-cochran/profi...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,306391,https://www.transfermarkt.com/charleston-batte...,4115,202102174115306391,phoenix rising_indy eleven_2021_2,aj cochran,1156285,29992,phoenix rising,27170,indy eleven,0,2021-02-17,2023-02-02,Transfer,0,EUR,,300553
1,2,aaren d'silva,20/21,2020-10-28,goa ii,goa,75000,NA,NA,https://www.transfermarkt.com/aaren-dsilva/pro...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,630382,https://www.transfermarkt.com/hyderabad-fc/sta...,76364,2020102876364630382,goa ii_goa_2020_10,aaren d'silva,1059236,61377,goa ii,31884,goa,0,2020-10-28,2021-06-18,Transfer,0,EUR,,703579
2,5,aaron amadi-holloway,21/22,2021-07-23,sc east bengal,burton albion,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2021072346842186574,sc east bengal_burton albion_2021_7,aaron amadi-holloway,1350346,15036,sc east bengal,1710,burton albion,0,2021-07-23,2022-01-28,Transfer,0,EUR,,10612
3,2,aaron amadi-holloway,22/23,2022-08-05,burton albion,hereford,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2022080546842186574,burton albion_hereford_2022_8,aaron amadi-holloway,1674586,1710,burton albion,36605,hereford,0,2022-08-05,2023-07-12,Transfer,0,EUR,,10612
4,16,aaron amadi-holloway,11/12,2011-11-24,bristol city,bath city,50000,NA,2012-06-30,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2011112446842186574,bristol city_bath city_2011_11,aaron amadi-holloway,80644,1655,bristol city,1728,bath city,0,2011-11-24,2012-01-01,Loan,0,,2011-11-26,10612
5,6,aaron amadi-holloway,20/21,2020-10-17,brisbane roar,sc east bengal,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2020101746842186574,brisbane roar_sc east bengal_2020_10,aaron amadi-holloway,1054694,8314,brisbane roar,15036,sc east bengal,0,2020-10-17,2021-07-23,Free Transfer,0,EUR,,10612
6,15,aaron amadi-holloway,11/12,2012-01-01,bath city,bristol city,50000,NA,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2012010146842186574,bath city_bristol city_2012_1,aaron amadi-holloway,1572374,1728,bath city,1655,bristol city,0,2012-01-01,2014-02-12,Back from Loan,0,EUR,,10612
7,6,aaron arnott,22/23,2023-01-13,brechin city,raith rovers,125000,NA,2023-05-31,https://www.transfermarkt.com/aaron-arnott/pro...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,822785,https://www.transfermarkt.com/brechin-city-fc/...,3194,202301133194822785,brechin city_raith rovers_2023_1,aaron arnott,1771045,8529,brechin city,8527,raith rovers,0,2023-01-13,2023-01-20,Transfer,0,EUR,,699833
8,3,aaron arnott,23/24,2023-09-19,raith rovers,east kilbride,125000,NA,2024-05-31,https://www.transfermarkt.com/aaron-arnott/pro...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_tr

In [41]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 231 entries, 0 to 250
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   row_ind             231 non-null    object             
 1   player_name         231 non-null    object             
 2   season              231 non-null    object             
 3   transfer_date       231 non-null    datetime64[ns]     
 4   team_from           231 non-null    object             
 5   team_to             231 non-null    object             
 6   market_value        231 non-null    object             
 7   transfer_value      231 non-null    object             
 8   contract_expiry     231 non-null    object             
 9   player_url          231 non-null    object             
 10  query_date          231 non-null    object             
 11  timestamp           231 non-null    datetime64[ns, UTC]
 12  blob                218 non-null    objec

In [42]:
# Funciones auxiliares para la normalización
def normalize_text(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        return ' '.join(text.lower().strip().split())
    return text

def simplify_name(name):
    parts = name.split()
    return ' '.join([parts[0], parts[-1]]) if len(parts) > 1 else name

def simplify_club_name(club_name):
    ignore_words = {'fc', 'club', 'de', 'the'}
    return ' '.join([word for word in club_name.lower().split() if word not in ignore_words])

# Normalizar y simplificar columnas relevantes
df1_sorted['player_name'] = df1_sorted['player_name'].apply(normalize_text).apply(simplify_name)
df1_sorted['team_from'] = df1_sorted['team_from'].apply(normalize_text).apply(simplify_club_name)
df1_sorted['team_to'] = df1_sorted['team_to'].apply(normalize_text).apply(simplify_club_name)
df1_sorted['transfer_date'] = pd.to_datetime(df1_sorted['transfer_date'], errors='coerce')

df2_sorted['nombre'] = df2_sorted['nombre'].apply(normalize_text).apply(simplify_name)
df2_sorted['fromTeamName'] = df2_sorted['fromTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted['toTeamName'] = df2_sorted['toTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted['startDate'] = pd.to_datetime(df2_sorted['startDate'], errors='coerce')

# Crear un ID temporal basado en mes y año
df1_sorted['ID'] = df1_sorted.apply(lambda row: f"{row['team_from']}_{row['team_to']}_{row['transfer_date'].year}_{row['transfer_date'].month}", axis=1)
df2_sorted['ID'] = df2_sorted.apply(lambda row: f"{row['fromTeamName']}_{row['toTeamName']}_{row['startDate'].year}_{row['startDate'].month}", axis=1)

# Primaria unión por ID
initial_merge = pd.merge(df1_sorted, df2_sorted, on='ID', suffixes=('_df1', '_df2'))

# Unión final basada en similitud
merged_rows = []

for _, merged_row in initial_merge.iterrows():
    name_similarity = fuzz.partial_ratio(merged_row['player_name'], merged_row['nombre'])
    if name_similarity > 80:
        merged_rows.append(merged_row)

# Crear un dataframe final con los resultados
merged_df3 = pd.DataFrame(merged_rows)

# Ver los resultados
print(merged_df3.head())

# Guardar el resultado en un archivo CSV
merged_df3.to_csv('archivo_unido3.csv', index=False)

  row_ind           player_name season transfer_date       team_from  \
0       3            aj cochran  20/21    2021-02-17  phoenix rising   
1       2        aakash sangwan  22/23    2022-06-21          punjab   
2       2         aaren d'silva  20/21    2020-10-28          goa ii   
4       5  aaron amadi-holloway  21/22    2021-07-23  sc east bengal   
5       2  aaron amadi-holloway  22/23    2022-08-05   burton albion   

         team_to market_value transfer_value contract_expiry  \
0    indy eleven       250000              0              NA   
1     chennaiyin       125000              0              NA   
2            goa        75000             NA              NA   
4  burton albion        2e+05              0              NA   
5       hereford        2e+05              0              NA   

                                          player_url  query_date  \
0  https://www.transfermarkt.com/aj-cochran/profi...  2024-09-10   
1  https://www.transfermarkt.com/aakash-sangwa

In [43]:
merged_df3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34063 entries, 0 to 58243
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   row_ind             34063 non-null  object             
 1   player_name         34063 non-null  object             
 2   season              34063 non-null  object             
 3   transfer_date       34063 non-null  datetime64[ns]     
 4   team_from           34063 non-null  object             
 5   team_to             34063 non-null  object             
 6   market_value        34063 non-null  object             
 7   transfer_value      34063 non-null  object             
 8   contract_expiry     34063 non-null  object             
 9   player_url          34063 non-null  object             
 10  query_date          34063 non-null  object             
 11  timestamp           34063 non-null  datetime64[ns, UTC]
 12  blob                31249 non-null  o

In [49]:
merged_df3.head(20)

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id,ID,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
0,3,aj cochran,20/21,2021-02-17,phoenix rising,indy eleven,250000,0,NA,https://www.transfermarkt.com/aj-cochran/profi...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,306391,https://www.transfermarkt.com/charleston-batte...,4115,202102174115306391,phoenix rising_indy eleven_2021_2,aj cochran,1156285,29992,phoenix rising,27170,indy eleven,0,2021-02-17,2023-02-02,Transfer,0,EUR,,300553
1,2,aakash sangwan,22/23,2022-06-21,punjab,chennaiyin,125000,0,NA,https://www.transfermarkt.com/aakash-sangwan/p...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,503167,https://www.transfermarkt.com/chennaiyin-fc/st...,45299,2022062145299503167,punjab_chennaiyin_2022_6,akash sangwan,1649519,37206,punjab,31882,chennaiyin,0,2022-06-21,2024-06-12,Transfer,0,EUR,,503194
2,2,aaren d'silva,20/21,2020-10-28,goa ii,goa,75000,NA,NA,https://www.transfermarkt.com/aaren-dsilva/pro...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,630382,https://www.transfermarkt.com/hyderabad-fc/sta...,76364,2020102876364630382,goa ii_goa_2020_10,aaren d'silva,1059236,61377,goa ii,31884,goa,0,2020-10-28,2021-06-18,Transfer,0,EUR,,703579
4,5,aaron amadi-holloway,21/22,2021-07-23,sc east bengal,burton albion,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2021072346842186574,sc east bengal_burton albion_2021_7,aaron amadi-holloway,1350346,15036,sc east bengal,1710,burton albion,0,2021-07-23,2022-01-28,Transfer,0,EUR,,10612
5,2,aaron amadi-holloway,22/23,2022-08-05,burton albion,hereford,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2022080546842186574,burton albion_hereford_2022_8,aaron amadi-holloway,1674586,1710,burton albion,36605,hereford,0,2022-08-05,2023-07-12,Transfer,0,EUR,,10612
6,16,aaron amadi-holloway,11/12,2011-11-24,bristol city,bath city,50000,NA,2012-06-30,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2011112446842186574,bristol city_bath city_2011_11,aaron amadi-holloway,80644,1655,bristol city,1728,bath city,0,2011-11-24,2012-01-01,Loan,0,,2011-11-26,10612
8,6,aaron amadi-holloway,20/21,2020-10-17,brisbane roar,sc east bengal,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2020101746842186574,brisbane roar_sc east bengal_2020_10,aaron amadi-holloway,1054694,8314,brisbane roar,15036,sc east bengal,0,2020-10-17,2021-07-23,Free Transfer,0,EUR,,10612
10,15,aaron amadi-holloway,11/12,2012-01-01,bath city,bristol city,50000,NA,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2012010146842186574,bath city_bristol city_2012_1,aaron amadi-holloway,1572374,1728,bath city,1655,bristol city,0,2012-01-01,2014-02-12,Back from Loan,0,EUR,,10612
13,6,aaron arnott,22/23,2023-01-13,brechin city,raith rovers,125000,NA,2023-05-31,https://www.transfermarkt.com/aaron-arnott/pro...,2024-09-08,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf..

In [46]:
import pandas as pd
import unicodedata
from fuzzywuzzy import fuzz

# Funciones auxiliares para la normalización
def normalize_text(text):
    if isinstance(text, str):
        text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
        return ' '.join(text.lower().strip().split())
    return text

def simplify_name(name):
    parts = name.split()
    return ' '.join([parts[0], parts[-1]]) if len(parts) > 1 else name

def simplify_club_name(club_name):
    ignore_words = {'fc', 'club', 'de', 'the'}
    return ' '.join([word for word in club_name.lower().split() if word not in ignore_words])

# Normalizar y simplificar columnas relevantes
df1_sorted['player_name'] = df1_sorted['player_name'].apply(normalize_text).apply(simplify_name)
df1_sorted['team_from'] = df1_sorted['team_from'].apply(normalize_text).apply(simplify_club_name)
df1_sorted['team_to'] = df1_sorted['team_to'].apply(normalize_text).apply(simplify_club_name)
df1_sorted['transfer_date'] = pd.to_datetime(df1_sorted['transfer_date'], errors='coerce')

df2_sorted['nombre'] = df2_sorted['nombre'].apply(normalize_text).apply(simplify_name)
df2_sorted['fromTeamName'] = df2_sorted['fromTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted['toTeamName'] = df2_sorted['toTeamName'].apply(normalize_text).apply(simplify_club_name)
df2_sorted['startDate'] = pd.to_datetime(df2_sorted['startDate'], errors='coerce')

# Crear un ID temporal basado en los nombres incluidos
df1_sorted['ID'] = df1_sorted.apply(lambda row: f"{row['player_name']}_{row['team_from']}_{row['team_to']}_{row['transfer_date'].year}_{row['transfer_date'].month}", axis=1)
df2_sorted['ID'] = df2_sorted.apply(lambda row: f"{row['nombre']}_{row['fromTeamName']}_{row['toTeamName']}_{row['startDate'].year}_{row['startDate'].month}", axis=1)

# Primaria unión por ID
initial_merge = pd.merge(df1_sorted, df2_sorted, on='ID', suffixes=('_df1', '_df2'))

# Unión final basada en similitud (ya no es muy necesaria si el ID incluye nombres)
merged_rows = []

for _, merged_row in initial_merge.iterrows():
    name_similarity = fuzz.partial_ratio(merged_row['player_name'], merged_row['nombre'])
    if name_similarity > 80:  # Esta parte podría omitirse si el ID ya diferencia lo suficiente
        merged_rows.append(merged_row)

# Crear un dataframe final con los resultados
merged_df4 = pd.DataFrame(merged_rows)

# Ver los resultados
print(merged_df4.head())

# Guardar el resultado en un archivo CSV
merged_df4.to_csv('archivo_unido4.csv', index=False)


  row_ind           player_name season transfer_date       team_from  \
0       3            aj cochran  20/21    2021-02-17  phoenix rising   
1       2         aaren d'silva  20/21    2020-10-28          goa ii   
2       5  aaron amadi-holloway  21/22    2021-07-23  sc east bengal   
3       2  aaron amadi-holloway  22/23    2022-08-05   burton albion   
4      16  aaron amadi-holloway  11/12    2011-11-24    bristol city   

         team_to market_value transfer_value contract_expiry  \
0    indy eleven       250000              0              NA   
1            goa        75000             NA              NA   
2  burton albion        2e+05              0              NA   
3       hereford        2e+05              0              NA   
4      bath city        50000             NA      2012-06-30   

                                          player_url  query_date  \
0  https://www.transfermarkt.com/aj-cochran/profi...  2024-09-10   
1  https://www.transfermarkt.com/aaren-dsilva/

In [48]:
merged_df4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31075 entries, 0 to 31074
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   row_ind             31075 non-null  object             
 1   player_name         31075 non-null  object             
 2   season              31075 non-null  object             
 3   transfer_date       31075 non-null  datetime64[ns]     
 4   team_from           31075 non-null  object             
 5   team_to             31075 non-null  object             
 6   market_value        31075 non-null  object             
 7   transfer_value      31075 non-null  object             
 8   contract_expiry     31075 non-null  object             
 9   player_url          31075 non-null  object             
 10  query_date          31075 non-null  object             
 11  timestamp           31075 non-null  datetime64[ns, UTC]
 12  blob                29283 non-null  o

In [50]:
merged_df4.head()

,row_ind,player_name,season,transfer_date,team_from,team_to,market_value,transfer_value,contract_expiry,player_url,query_date,timestamp,blob,player_id,teams_url,team_id,player_transfer_id,ID,nombre,transferId,fromTeamId,fromTeamName,toTeamId,toTeamName,active,startDate,endDate,type,value,currency,announceDate,playerId
0,3,aj cochran,20/21,2021-02-17,phoenix rising,indy eleven,250000,0,NA,https://www.transfermarkt.com/aj-cochran/profi...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,306391,https://www.transfermarkt.com/charleston-batte...,4115,202102174115306391,aj cochran_phoenix rising_indy eleven_2021_2,aj cochran,1156285,29992,phoenix rising,27170,indy eleven,0,2021-02-17,2023-02-02,Transfer,0,EUR,,300553
1,2,aaren d'silva,20/21,2020-10-28,goa ii,goa,75000,NA,NA,https://www.transfermarkt.com/aaren-dsilva/pro...,2024-09-10,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,630382,https://www.transfermarkt.com/hyderabad-fc/sta...,76364,2020102876364630382,aaren d'silva_goa ii_goa_2020_10,aaren d'silva,1059236,61377,goa ii,31884,goa,0,2020-10-28,2021-06-18,Transfer,0,EUR,,703579
2,5,aaron amadi-holloway,21/22,2021-07-23,sc east bengal,burton albion,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2021072346842186574,aaron amadi-holloway_sc east bengal_burton alb...,aaron amadi-holloway,1350346,15036,sc east bengal,1710,burton albion,0,2021-07-23,2022-01-28,Transfer,0,EUR,,10612
3,2,aaron amadi-holloway,22/23,2022-08-05,burton albion,hereford,2e+05,0,NA,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2022080546842186574,aaron amadi-holloway_burton albion_hereford_20...,aaron amadi-holloway,1674586,1710,burton albion,36605,hereford,0,2022-08-05,2023-07-12,Transfer,0,EUR,,10612
4,16,aaron amadi-holloway,11/12,2011-11-24,bristol city,bath city,50000,NA,2012-06-30,https://www.transfermarkt.com/aaron-amadi-holl...,2024-09-09,2024-09-07 14:27:45+00:00,transfermarkt/player_transfers/ players_transf...,186574,https://www.transfermarkt.com/fc-hereford/star...,46842,2011112446842186574,aaron amadi-holloway_bristol city_bath city_20...,aaron amadi-holloway,80644,1655,bristol city,1728,bath city,0,2011-11-24,2012-01-01,Loan,0,,2011-11-26,10612


DOCU FUZZY  https://pypi.org/project/fuzzywuzzy/

In [53]:
fuzz.ratio("this is a test", "this is a test!")

97

In [54]:
fuzz.partial_ratio("this is a test", "this is a test!")
 

100

In [55]:
fuzz.token_sort_ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear")

100

In [57]:
fuzz.token_sort_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")

84

In [58]:
fuzz.token_set_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")

100